## Text Analysis - Prepare Tokenized (Text) Corpus
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [15]:
%load_ext autoreload
%autoreload 2

import os

import text_analytic_tools.notebooks_gui.load_corpus_gui as load_corpus_gui
import text_analytic_tools.common.textacy_utility as textacy_utility
import text_analytic_tools.utility as utility
import text_analytic_tools.notebooks_gui.extract_corpus_text_gui as extract_corpus_text_gui

from text_analytic_tools.domain_config import current_domain as domain_logic

from beakerx.object import beakerx
from beakerx import *
from IPython.display import display

logger = utility.getLogger('corpus_text_analysis')

current_corpus_container = lambda: textacy_utility.CorpusContainer.container()
current_corpus = lambda: textacy_utility.CorpusContainer.corpus()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <span style='color: green'>PREPARE </span> Load and Prepare Corpus <span style='float: right; color: red'>MANDATORY</span>


In [8]:

try:
    container = current_corpus_container()
    load_corpus_gui.display_corpus_load_gui(domain_logic.DATA_FOLDER, container=container, domain_logic=domain_logic)
except Exception as ex:
    raise
    logger.error(ex)


## <span style='color: green'>PREPARE </span> Extract Text From Corpus <span style='float: right; color: green'>TRY IT</span>

In [11]:

def extract_token_callback(corpus, corpus_path, **opts):

    docs = ((doc._.meta['filename'], doc) for doc in corpus)
    term_substitutions = domain_logic.term_substitutions(corpus.spacy_lang.vocab)

    tokenized_docs = textacy_utility.extract_document_tokens(docs, term_substitutions=term_substitutions, **opts)

    filepath, df_summary = common.store_tokenized_corpus_as_archive(tokenized_docs, corpus_path)

    logger.info("DONE! Result stored in '{}'".format(filepath))

    display(df_summary)

try:

    corpus = current_corpus_container().textacy_corpus
    corpus_path = current_corpus_container().prepped_source_path

    gui = extract_corpus_text_gui.build(corpus, corpus_path, domain_logic.pos_tags(), extract_token_callback)

    display(gui.boxes)

except Exception as ex:
    logger.error(ex)
    

2019-10-26 20:38:16,947 : ERROR : 'NoneType' object is not iterable
